In [1]:
# High Dynamic Range (HDR) 
# https://docs.opencv.org/3.1.0/d2/df0/tutorial_py_hdr.html

In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt

In [3]:
cd ../data

C:\Users\Manuel\Dropbox\docencia vision 2018\desafio 10\data


In [4]:
# Loading exposure images into a list
img_fn = ["StLouisArchMultExpImg0.jpg", "StLouisArchMultExpImg1.jpg", "StLouisArchMultExpImg2.jpg", "StLouisArchMultExpImg3.jpg"]
img_list = [cv2.imread(fn) for fn in img_fn]
exposure_times = np.array([15.0, 2.5, 0.25, 0.0333], dtype=np.float32)

In [5]:
# Merge exposures to HDR image
merge_debvec = cv2.createMergeDebevec()
hdr_debvec = merge_debvec.process(img_list, times=exposure_times.copy())
merge_robertson = cv2.createMergeRobertson()
hdr_robertson = merge_robertson.process(img_list, times=exposure_times.copy())

In [6]:
 # Tonemap HDR image
tonemap1 = cv2.createTonemapDurand(gamma=2.2)
res_debvec = tonemap1.process(hdr_debvec.copy())
tonemap2 = cv2.createTonemapDurand(gamma=1.3)
res_robertson = tonemap2.process(hdr_robertson.copy())

In [7]:
# Exposure fusion using Mertens
merge_mertens = cv2.createMergeMertens()
res_mertens = merge_mertens.process(img_list)

In [8]:
# Convert datatype to 8-bit and save
res_debvec_8bit = np.clip(res_debvec*255, 0, 255).astype('uint8')
res_robertson_8bit = np.clip(res_robertson*255, 0, 255).astype('uint8')
res_mertens_8bit = np.clip(res_mertens*255, 0, 255).astype('uint8')

cv2.imwrite("output_ldr_debvec.jpg", res_debvec_8bit)
cv2.imwrite("output_ldr_robertson.jpg", res_robertson_8bit)
cv2.imwrite("output_fusion_mertens.jpg", res_mertens_8bit)

True

In [9]:
# Estimating Camera Response Function 

In [10]:
# Estimate camera response function (CRF)
cal_debvec = cv2.createCalibrateDebevec()
crf_debvec = cal_debvec.process(img_list, times=exposure_times)
hdr_debvec = merge_debvec.process(img_list, times=exposure_times.copy(), response=crf_debvec.copy())
cal_robertson = cv2.createCalibrateRobertson()
crf_robertson = cal_robertson.process(img_list, times=exposure_times)
hdr_robertson = merge_robertson.process(img_list, times=exposure_times.copy(), response=crf_robertson.copy())

In [11]:
hdr_debvec.shape

(2112, 2816, 3)